## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-12-19-22-48-37 +0000,nypost,Jeffrey Epstein pictured with pop legend and a...,https://nypost.com/2025/12/19/us-news/jeffrey-...
1,2025-12-19-22-47-30 +0000,nypost,US launches ‘massive strike’ on ISIS in Syria ...,https://nypost.com/2025/12/19/us-news/us-launc...
2,2025-12-19-22-47-19 +0000,nyt,Scott C. Beardsley Named University of Virgini...,https://www.nytimes.com/2025/12/19/us/uva-pres...
3,2025-12-19-22-42-55 +0000,bbc,US to strip alleged Bosnian war criminal of ci...,https://www.bbc.com/news/articles/c2k422e7ykvo...
4,2025-12-19-22-42-06 +0000,latimes,California sees population growth for third co...,https://www.latimes.com/california/story/2025-...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2362/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
70,trump,42
26,new,19
76,ukraine,18
196,shooting,17
540,bondi,15


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
363,2025-12-18-23-26-52 +0000,nypost,GOP senators push Trump to use $5 billion in R...,https://nypost.com/2025/12/18/world-news/gop-s...,117
65,2025-12-19-21-06-58 +0000,nypost,DOJ releases thousands of Jeffrey Epstein case...,https://nypost.com/2025/12/19/us-news/doj-rele...,94
77,2025-12-19-20-31-32 +0000,nypost,Trump says he will haul in ‘rich’ health insur...,https://nypost.com/2025/12/19/us-news/trump-sa...,91
315,2025-12-19-03-19-00 +0000,wsj,President Trump signed into law new powers to ...,https://www.wsj.com/politics/national-security...,85
69,2025-12-19-20-47-48 +0000,nyt,Trump Administration Pauses Green Card Lottery...,https://www.nytimes.com/2025/12/19/us/brown-sh...,85


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
363,117,2025-12-18-23-26-52 +0000,nypost,GOP senators push Trump to use $5 billion in R...,https://nypost.com/2025/12/18/world-news/gop-s...
293,80,2025-12-19-05-54-00 +0000,wsj,Brown and MIT Shooting Suspect Found Dead in N...,https://www.wsj.com/us-news/brown-shooting-sus...
32,79,2025-12-19-22-09-00 +0000,wsj,The Justice Department has released files rela...,https://www.wsj.com/politics/policy/justice-de...
281,55,2025-12-19-08-34-47 +0000,nyt,Australia Announces Plan to Buy Back Guns in W...,https://www.nytimes.com/2025/12/19/world/asia/...
39,51,2025-12-19-22-00-00 +0000,wsj,"Rep. Elise Stefanik (R., N.Y.) said she would ...",https://www.wsj.com/politics/elections/elise-s...
23,50,2025-12-19-22-21-47 +0000,nypost,Elon Musk wins appeal to restore $56B Tesla pa...,https://nypost.com/2025/12/19/business/elon-mu...
77,49,2025-12-19-20-31-32 +0000,nypost,Trump says he will haul in ‘rich’ health insur...,https://nypost.com/2025/12/19/us-news/trump-sa...
26,43,2025-12-19-22-17-50 +0000,missionlocal,Nightmare before Christmas: Fillmore low-incom...,https://missionlocal.org/2025/12/sf-thomas-pai...
338,40,2025-12-19-00-54-18 +0000,nypost,Erika Kirk vows to get JD Vance elected presid...,https://nypost.com/2025/12/18/us-news/erika-ki...
194,37,2025-12-19-15-14-00 +0000,wsj,Home sales rose in November for the third stra...,https://www.wsj.com/economy/housing/home-sales...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
